In [1]:
import functools
import os

import clu.data.dataset_iterator
import tensorflow as tf
import jax
from jax import random
from jax.experimental import multihost_utils
import jax.numpy as jnp
from flax import linen
import numpy as np
import seqio
import t5.data
from t5.evaluation import metrics as t5_metrics

import t5x
from t5x import partitioning
from t5x import train_state as train_state_lib
from t5x import utils
from t5x.examples.decoder_only import network as decoder_only_network
from t5x.examples.t5 import network
from t5x.examples.scalable_t5 import network as scalable_network
from t5x.interactive_model import InteractiveModel
from t5x.interactive_model import get_batches_from_seqio
from t5x.interactive_model import InferenceType
import nest_asyncio
nest_asyncio.apply()

2023-04-16 16:20:43.377064: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2023-04-16 16:20:44.005930: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2023-04-16 16:20:44.006035: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2023-04-16 16:20:44.006045: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use

In [2]:
input_vocabulary=seqio.SentencePieceVocabulary(sentencepiece_model_file="gs://n2formal-public-data-europe/albert/tokenizer/galactica_enhanced.model")
output_vocabulary=seqio.SentencePieceVocabulary(sentencepiece_model_file="gs://n2formal-public-data-europe/albert/tokenizer/galactica_enhanced.model")
optimizer=t5x.adafactor.Adafactor(decay_rate=0.8, step_offset=0, logical_factor_rules=t5x.adafactor.standard_logical_factor_rules())
decode_fn=functools.partial(t5x.decoding.temperature_sample, temperature=1.0, topk=40)

# Define a model using the minimal T5 module.
t5_module = decoder_only_network.Decoder(
    config=decoder_only_network.TransformerConfig(
        vocab_size=32128,
        dtype='bfloat16',
        emb_dim=768,
        num_heads=12,
        num_layers=12,
        head_dim=64,
        mlp_dim=2048,
        mlp_activations=('gelu', 'linear'),
        dropout_rate=0.0,
        logits_via_embedding=True
    )
)
model = t5x.models.DecoderOnlyModel(
    module=t5_module,
    vocabulary=input_vocabulary,
    optimizer_def=optimizer,
    decode_fn=decode_fn
)

In [3]:
# # Define EncoderDecoderModel constructor args (except the module).
# input_vocabulary=t5.data.get_default_vocabulary()
# output_vocabulary=t5.data.get_default_vocabulary()
# optimizer=t5x.adafactor.Adafactor(decay_rate=0.8, step_offset=0, logical_factor_rules=t5x.adafactor.standard_logical_factor_rules())
# decode_fn=functools.partial(t5x.decoding.temperature_sample, temperature=1.0, topk=40)

# # Define a model using the minimal T5 module.
# t5_module = network.Transformer(config=network.T5Config(
#     vocab_size=32128,
#     dtype='bfloat16',
#     emb_dim=512,
#     num_heads=6,
#     num_encoder_layers=8,
#     num_decoder_layers=8,
#     head_dim=64,
#     mlp_dim=1024,
#     mlp_activations=('gelu', 'linear'),
#     dropout_rate=0.0,
#     logits_via_embedding=False))
# model = t5x.models.EncoderDecoderModel(
#     module=t5_module,
#     input_vocabulary=input_vocabulary,
#     output_vocabulary=output_vocabulary,
#     optimizer_def=optimizer,
#     decode_fn=decode_fn)

In [4]:
# The checkpoint below is a T5-1.1-Small checkpoint (https://github.com/google-research/t5x/blob/main/docs/models.md) 
# that has additionally been finetuned on the (Open Domain) Natural Questions 
# benchmark (https://ai.google.com/research/NaturalQuestions).
# checkpoint_path='gs://t5-data/pretrained_models/cbqa/small_ssm_nq/model.ckpt-1110000'
checkpoint_path='gs://n2formal-public-data-europe/albert/model_checkpoints/pretrain_lovelace_decoder_base_default/checkpoint_90000/checkpoint'
dtype='bfloat16'

restore_mode='specific'

In [5]:
partitioner=partitioning.PjitPartitioner(
        num_partitions=1,
        model_parallel_submesh=None)

In [6]:
batch_size=8
task_feature_lengths = {'inputs': 256, 'targets': 256}
output_dir='/tmp/output_dir'
input_shapes = {
    'encoder_input_tokens': np.array([8, 256]),
    'decoder_target_tokens': np.array([8, 256]),
    'decoder_input_tokens': np.array([8, 256]),
    'decoder_loss_weights': np.array([8, 256])
}

interactive_model = InteractiveModel(
  batch_size=batch_size,
  task_feature_lengths=task_feature_lengths,
  output_dir=output_dir,
  partitioner=partitioner,
  model=model,
  dtype=dtype,
  restore_mode=restore_mode,
  checkpoint_path=checkpoint_path,
  input_shapes=input_shapes
)

dict_keys(['state', 'target'])
dict_keys(['param_states', 'step'])
dict_keys(['decoder'])
Hooray!
dict_keys(['decoder_norm', 'layers_0', 'layers_1', 'layers_10', 'layers_11', 'layers_2', 'layers_3', 'layers_4', 'layers_5', 'layers_6', 'layers_7', 'layers_8', 'layers_9', 'token_embedder'])
dict_keys(['decoder_norm', 'layers_0', 'layers_1', 'layers_10', 'layers_11', 'layers_2', 'layers_3', 'layers_4', 'layers_5', 'layers_6', 'layers_7', 'layers_8', 'layers_9', 'token_embedder'])
Hooray!
dict_keys(['decoder_norm', 'layers_0', 'layers_1', 'layers_10', 'layers_11', 'layers_2', 'layers_3', 'layers_4', 'layers_5', 'layers_6', 'layers_7', 'layers_8', 'layers_9', 'token_embedder'])
dict_keys(['decoder_norm', 'layers_0', 'layers_1', 'layers_10', 'layers_11', 'layers_2', 'layers_3', 'layers_4', 'layers_5', 'layers_6', 'layers_7', 'layers_8', 'layers_9', 'token_embedder'])


I0000 00:00:1681662057.014130  887136 gcs_resource.cc:102] Using default AdmissionQueue with limit 32
I0000 00:00:1681662057.017613  888218 google_auth_provider.cc:179] Running on GCE, using service account 398558348766-compute@developer.gserviceaccount.com
2023-04-16 16:21:05.364077: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2023-04-16 16:21:05.364120: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)


In [7]:
question = 'what is the capital of france?'

In [8]:
len(input_vocabulary.encode(question))

8

In [9]:
validation_examples = [
  {
      'target': 'Paris', 
      'input': "Where is the capitcal of France?"
      },
  {
      'target': 'Rome', 
      'input': "Where is the capitcal of France?"
      },
  {
      'target': 'Moscow', 
      'input': "Where is the capitcal of France?"
      },
  {
      'target': 'London', 
      'input': "Where is the capitcal of France?"
      },
  {
      'target': 'Beijing', 
      'input': "Where is the capitcal of France?"
      },
  {
      'target': 'Tokyo', 
      'input': "Where is the capitcal of France?"
      },
  {
      'target': 'Jakarta', 
      'input': "Where is the capitcal of France?"
      },
  {
      'target': 'Liverpool', 
      'input': "lemma \"1+2=3\" by auto"
      },
]

In [10]:
examples_and_predictions, _ = interactive_model.predict_with_aux(examples=validation_examples)
predictions = [prediction for example, prediction in examples_and_predictions]

examples_and_scores = interactive_model.score(examples=validation_examples)
scores = [score for example, score in examples_and_scores]
for val_example, prediction, score in zip(validation_examples, predictions, scores):
    print(f"Input: {val_example['input']}")
    print(f"Prediction: {prediction.decode('utf-8')}")
    print(f"Target: {val_example['target']}")
    print(f"Score: {score}")

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Input: Where is the capitcal of France?
Prediction: Sir Avraham Lehdet ghet, or 'L', or 'F'? That we s'il or two would be best used as a stand on 'T'. You will find that a certain note _t.it_ would come from the house in the centre of England while we could find it. 'T's what ?' 'T's the place d'un petit d'un air-star haste scythes. The room you go into is the room, its apricots which the room will be called: 'T, the house!' 'T!?' 'T!' 'T!' The rooms are a bit smaller to make things work. A few dot scythes and scythes, scythes and one is cyth scythe! The cyth is cyth but all cyth is the place. There is a small cyth scythe. An excellent d'fore it will be to have a large cyth in it. When you are ready to go, look, and look, try the next door. You can scythe _?_ It will come later, a little further. 'T!-----------------------------------------------------------------------------' p. 92. 'L', which ?? 's been snubbed by the lady of the court, is so much so that '---------------------------